In [1]:
# Import necessary libraries
from transformers import T5ForConditionalGeneration, T5Tokenizer, Trainer, TrainingArguments
from datasets import load_dataset

2023-04-26 21:57:10.287427: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-26 21:57:10.621856: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-04-26 21:57:10.621887: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-04-26 21:57:11.678389: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [2]:
# Load XSum dataset
dataset = load_dataset('xsum', split='train[:10]')

Found cached dataset xsum (/home/volt/.cache/huggingface/datasets/xsum/default/1.2.0/082863bf4754ee058a5b6f6525d0cb2b18eadb62c7b370b095d1364050a52b71)


In [3]:
# Define the models and tokenizer
model1 = T5ForConditionalGeneration.from_pretrained('t5-base')
tokenizer1 = T5Tokenizer.from_pretrained('t5-base')

model2 = T5ForConditionalGeneration.from_pretrained('t5-small')
tokenizer2 = T5Tokenizer.from_pretrained('t5-small')

model3 = T5ForConditionalGeneration.from_pretrained('t5-large')
tokenizer3 = T5Tokenizer.from_pretrained('t5-large')

/home/volt/.local/lib/python3.10/site-packages/transformers/models/t5/tokenization_t5.py:163: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [ ]:
# Define the training arguments
training_args = TrainingArguments(
    output_dir='../results',
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=1,
    weight_decay=0.01,
    push_to_hub=False,
    logging_dir='./logs',
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model='rouge2',
    greater_is_better=True
)

In [ ]:
# Define the trainer objects
trainer1 = Trainer(
    model=model1,
    args=training_args,
    train_dataset=dataset,
    data_collator=lambda data: {'input_ids': tokenizer1(data['document'], truncation=True, padding='max_length')['input_ids'], 
                                'attention_mask': tokenizer1(data['document'], truncation=True, padding='max_length')['attention_mask'], 
                                'labels': tokenizer1(data['summary'], truncation=True, padding='max_length')['input_ids']}
)

trainer2 = Trainer(
    model=model2,
    args=training_args,
    train_dataset=dataset,
    data_collator=lambda data: {'input_ids': tokenizer2(data['document'], truncation=True, padding='max_length')['input_ids'], 
                                'attention_mask': tokenizer2(data['document'], truncation=True, padding='max_length')['attention_mask'], 
                                'labels': tokenizer2(data['summary'], truncation=True, padding='max_length')['input_ids']}
)

trainer3 = Trainer(
    model=model3,
    args=training_args,
    train_dataset=dataset,
    data_collator=lambda data: {'input_ids': tokenizer3(data['document'], truncation=True, padding='max_length')['input_ids'], 
                                'attention_mask': tokenizer3(data['document'], truncation=True, padding='max_length')['attention_mask'], 
                                'labels': tokenizer3(data['summary'], truncation=True, padding='max_length')['input_ids']}
)

In [ ]:
# Train the models
trainer1.train()
trainer2.train()
trainer3.train()